# Object-oriented asset file system

In [ ]:
import ee, geetools
from geetools.utils import initialize_documentation

initialize_documentation()

[![github](https://img.shields.io/badge/-see%20sources-white?logo=github&labelColor=555)](https://github.com/gee-community/geetools/blob/main/docs/usage/asset.ipynb)
[![colab](https://img.shields.io/badge/-open%20in%20colab-blue?logo=googlecolab&labelColor=555)](https://colab.research.google.com/github/gee-community/geetools/blob/main/docs/usage/asset.ipynb)

## Set up environment

Install all the required libs if necessary and perform the import statements upstream.

In [ ]:
# uncomment if installation of libs is necessary
# !pip install earthengine-api geetools

In [ ]:
import ee
import geetools #noqa: F401

In [ ]:
# uncomment if authetication to GEE is needed
# ee.Authenticate()

In [ ]:
# uncomment if initialization is required
# ee.Initialize()

## The `Asset` object

In Google Earth Engine API, users are working with Assets. An asset is a filelike object that englobes a wide variety of types: IMAGE, IMAGE_COLLECTION, FOLDER, TABLE, FEATURE_COLLECTION, etc.

They are identified by a unique ID, which is a string that looks like: `projects/username/assets/foo`. Using the vanila Earthengine API, They can be modified using the `ee.data` module. This module has been proven complicated when dealing with basic file manipulation operation such as listing, moving, copying, etc.

`geetools` provides a simple way to manage assets as an object-oriented filesystem paths using the `Asset` object. This object is a subclass of the `pathlib.Path` object, which is a powerful way to manage file paths in Python. Most of the methods and properties are overwritten to work with the Google Earth Engine context.

`ee.Asset` objects implement the os.PathLike interface, allowing them to be used anywhere the interface is accepted.

## Basic use

Importing the main class:

In [ ]:
import ee, geetools

### Create asset objects 

The Asset objects etend the pathlib.Path object and thus behave exactly the same when dealing with constructor. THe only differnece is that asset path only supports posix-like file separator: `/`.

In [ ]:
ee.Asset("projects/ee-geetools/assets/documentation/image1")

Each element of pathsegments can be either a string representing a path segment, or an object implementing the os.PathLike interface where the __fspath__() method returns a string, such as another path object.

In [ ]:
ee.Asset("projects", "ee-geetools", "assets", "documentation", "image1")

In [ ]:
ee.Asset("projects/ee-geetools/assets/documentation") / "image1"

In [ ]:
ee.Asset("projects/ee-geetools/assets/documentation").joinpath("image1")

### Listing subdirectories

In [ ]:
# a public folder created for this docuemntation
folder = ee.Asset("projects/ee-geetools/assets/documentation")

# list all its direct subdirectories
[a for a in folder.iterdir() if a.is_folder()]

```{api}
- {docstring}`ee.Asset.iterdir`
- {{docstring}`ee.Asset.is_folder`
```

### Listing Image in this folder

In [ ]:
[a for a in folder.iterdir() if a.is_image()]

In [ ]:
[a for a in folder.glob("**/image*")]

```{api}
- {docstring}`ee.Asset.iterdir`
- {docstring}`ee.Asset.glob`
- {docstring}`ee.Asset.is_image`
```

### Querying asset properties

In [ ]:
folder.exists()

In [ ]:
fakeImage = folder / "image6"
fakeImage.exists()

```{api}
- {docstring}`ee.Asset.exists`
```

## General properties

Paths are immutable and hashable. Paths of a same flavour are comparable and orderable. These properties respect the flavour’s case-folding semantics:

In [ ]:
folder = ee.Asset("projects/ee-geetools/assets/documentation")

In [ ]:
folder == ee.Asset("projects/ee-geetools/assets/DOCUMENTATION")

In [ ]:
folder in { ee.Asset("projects/ee-geetools/assets/documentation")}

The slash operator helps create child asset, like `os.path.join()`. If the argument is an absolute asset, the previous path is ignored.

In [ ]:
ee.Asset("projects/ee-geetools/assets/documentation") / "image1"

An asset object can be used anywhere an object implementing `os.PathLike` is accepted.

In [ ]:
import os

a = ee.Asset("projects/ee-geetools/assets/documentation")
os.fspath(a)

The string representation of an asset is the asset id itself, which you can pass to any function taking an asset id as a string:

In [ ]:
a = ee.Asset("projects/ee-geetools/assets/documentation/image1")
str(a)

### Accessing individual parts

To access the individual “parts” (components) of a path, use the following property:

In [ ]:
a = ee.Asset("projects/ee-geetools/assets/documentation/image1")
a.parts

```{api}
- {docstring}`ee.Asset.parts`
```

### access parent container

Asset parent containers can be access either by the `parent` property or the `parents` property. Note This is a purely lexical operation and the parent is not checked to exist.

```{api}
- {docstring}`ee.Asset.parent`
- {docstring}`ee.Asset.parents`
```

In [ ]:
a = ee.Asset("projects/ee-geetools/assets/documentation/subfolder1/image1")
a.parent

In [ ]:
a = ee.Asset("projects/ee-geetools/assets/documentation/subfolder1/image1")
a.parents

### Name of the asset

A string representing the final path component can be used to get the name of the asset.add

```{api}
- {docstring}`ee.Asset.name`
```

In [ ]:
a = ee.Asset("projects/ee-geetools/assets/documentation/subfolder1/image1")
a.name

## General Methods

Pure paths provide the following methods.

### evaluate relation between assets

It's possible to check if files are related between one another using the following methods:

```{api}
- {docstring}`ee.Asset.is_relative_to`

In [ ]:
a = ee.Asset("projects/ee-geetools/assets/documentation/subfolder1/image1")
b = ee.Asset("projects/ee-geetools/assets/documentation")
a.is_relative_to(b)

### create a siblings

One can create a siblings asset in the same container by using the `with_name()` method:

```{api}    
- {docstring}`ee.Asset.with_name`
```

In [ ]:
a = ee.Asset("projects/ee-geetools/assets/documentation/subfolder1/image1")
a.with_name("image2")

### resolve unix like symbols

One can use some unix-like descriptors in it's Asset constructor parameters. If so before using the Asset object, it is necessary to resolve these symbols. The method `expanduser` does that.

```{api}
- {py:meth}`expanduser <ee.Asset.Asset.expanduser>`: {docstring}`ee.Asset.expanduser`
```

In [ ]:
a = ee.Asset("~/documentation/subfolder1/image1")
a.expanduser()

### check existence

One can check if an asset exists using the `exists` method:

```{api}
- {py:meth}`exists <ee.Asset.Asset.exists>`: {docstring}`ee.Asset.exists`
```

In [ ]:
a = ee.Asset("projects/ee-geetools/assets/documentation/subfolder1/image1")
a.exists()

In [ ]:
a = ee.Asset("projects/ee-geetools/assets/documentation/subfolder1/image10")
a.exists()

### Evaluate asset type

As Earth Engine is not using any file extention to differentiate the asset type, one can use the `is_type` method with any of the following types: `IMAGE`, `IMAGE_COLLECTION`, `FOLDER`, `TABLE`, `FEATURE_COLLECTION`, `UNKNOWN`.

```{api}
- {py:meth}`is_type <ee.Asset.Asset.is_type>`: {docstring}`ee.Asset.is_type`
```

In [ ]:
a = ee.Asset("projects/ee-geetools/assets/documentation/subfolder1/image1")
a.is_type("IMAGE")

All type checks are available in dedicated wrapped methods like `is_image`, `is_folder`, `is_table` ...etc.

In [ ]:
a.is_image()

Many other useful methods are available and are described in the {py:class}`API documentation <ee.Asset.Asset>`.